In [1]:
from requests import get
from json import loads
import pandas as pd

URL = "https://api.warframe.market/v1/"



# Getting item information from the warframe.market API

In [14]:
r = get("https://api.warframe.market/v1/items")
r.status_code

200

In [15]:
d = dict(loads(r.content))


In [16]:
# something that is going to have to be done with all of the responses from the api
# all of the responses are held ina  payload key that has to be removed
# easiest way of doing that is setting it equal to the value contained within payload
d = d['payload']

# the various items are held in a list of dictionaries, that list's key is `items`
# we are going to remove that so i can get the list of entries
d = d['items']

In [17]:
# this is going to be used by my search feature, iter the dict keys and get O(1) lookup for items by name geting the results to pop up is going to ge interesting

q = {}
for entry in d:
    # store each item dictionary in a dict by the items name
    q[entry['item_name']] = entry

# look at the head of q
[x for y, x in enumerate(q) if y < 5]

#self.items_dict = q # remove when incorporated into oop class

['Axi C1 Intact',
 'Meso S3 Intact',
 'Lith N2 Exceptional',
 'Lith N3 Exceptional',
 'Neo V4 Intact']

In [7]:

# testing out grabbing an items description
r = get(URL+"items/lith_n2_exceptional")
r.status_code

200

In [9]:
d = dict(loads(r.content))['payload']
d = d['item']
d['items_in_set'][0]['es']

# @TODO add wiki lookup with `wiki_link`

# @TODO add scraped drop information to this part of the data

{'item_name': 'Lith N2 Exceptional',
 'description': '<p>Void Relics are Orokin objects that can be opened to reveal valuable treasure enclosed within by completing Void Fissure missions. They are the principal means of acquiring Prime and Forma Blueprints.</p>',
 'wiki_link': 'http://warframe.wikia.com/wiki/Void_Relic',
 'drop': []}

In [10]:
desc = d['items_in_set'][0]['es']['description'].replace('<p>','').replace('</p>','')
desc

'Void Relics are Orokin objects that can be opened to reveal valuable treasure enclosed within by completing Void Fissure missions. They are the principal means of acquiring Prime and Forma Blueprints.'

In [12]:

# cleanup
del q, d, desc, entry

NameError: name 'q' is not defined

# getting item market stats

In [18]:
item = q['Equinox Prime Systems']
r = get(URL + f"items/{item['url_name']}/statistics")



In [19]:
d = dict(loads(r.content))['payload']
d['statistics_live'].keys()

dict_keys(['48hours', '90days'])

In [20]:
d['statistics_live']['48hours'][0]

{'datetime': '2020-09-07T02:00:00.000+00:00',
 'volume': 142,
 'min_price': 5.0,
 'max_price': 13.0,
 'avg_price': 9.0,
 'wa_price': 7.718,
 'median': 7.0,
 'order_type': 'buy',
 'moving_avg': 12.2,
 'id': '5f55a274db955400242d1660'}

In [21]:
two_day = d['statistics_live']['48hours']

# print a list of 5 order_id's
for i in range(len(two_day)):
    if i < 5:
        print(two_day[i]['id'])
    else:
        break
     

5f55a274db955400242d1660
5f55a274db955400242d165f
5f55b08edb955400272c9c9c
5f55b08edb955400272c9c9b
5f55be9fdb955400252d3149



# scrapping live market orders

In [125]:
r = get(URL + "most_recent")

In [126]:
print(r.status_code)
r.content


200


b'{"payload": {"sell_orders": [{"visible": true, "region": "en", "platform": "pc", "user": {"region": "en", "reputation": 0, "ingame_name": "GsouzaA", "status": "ingame", "avatar": null, "id": "5d9ea6e3b6afee07454f3985"}, "order_type": "sell", "creation_date": "2020-08-13T00:42:57.033+00:00", "last_update": "2020-08-13T00:42:57.033+00:00", "item": {"url_name": "soma_prime_set", "pt": {"item_name": "Soma Prime Set"}, "icon": "icons/en/Soma_Prime_Set.1bf4937a6c2106c373cffeb93b1c87e0.png", "en": {"item_name": "Soma Prime Set"}, "sub_icon": null, "ru": {"item_name": "\\u0421\\u043e\\u043c\\u0430 \\u043f\\u0440\\u0430\\u0439\\u043c \\u041a\\u043e\\u043c\\u043f\\u043b\\u0435\\u043a\\u0442"}, "fr": {"item_name": "Soma Prime Set"}, "es": {"item_name": "Soma Prime Set"}, "zh": {"item_name": "Soma Prime Set"}, "tags": ["set", "prime"], "thumb": "icons/en/thumbs/Soma_Prime_Set.1bf4937a6c2106c373cffeb93b1c87e0.128x128.png", "pl": {"item_name": "Soma Prime Set"}, "sv": {"item_name": "Soma Prime Set

In [131]:
pd.DataFrame(dict(loads(r.content))['payload']['sell_orders'])


,visible,region,platform,user,order_type,creation_date,last_update,item,platinum,quantity,id,mod_rank
0,True,en,pc,"{'region': 'en', 'reputation': 0, 'ingame_name...",sell,2020-08-13T00:42:57.033+00:00,2020-08-13T00:42:57.033+00:00,"{'url_name': 'soma_prime_set', 'pt': {'item_na...",45.0,1,5f348c9148f69d018b0bed8d,NaN
1,True,en,pc,"{'region': 'en', 'reputation': 1, 'ingame_name...",sell,2020-08-13T00:42:55.908+00:00,2020-08-13T00:42:55.908+00:00,"{'url_name': 'dethcube_prime_blueprint', 'pt':...",10.0,1,5f348c8fad961e01bd2d3b3c,NaN
2,True,en,pc,"{'region': 'en', 'reputation': 2, 'ingame_name...",sell,2020-08-13T00:42:53.436+00:00,2020-08-13T00:42:53.436+00:00,"{'url_name': 'decisive_judgement', 'pt': {'ite...",5.0,3,5f348c8d46b8f1015b8a26a7,0.0
3,True,en,pc,"{'region': 'en', 'reputation': 1, 'ingame_name...",sell,2020-08-13T00:42:44.483+00:00,2020-08-13T00:42:44.483+00:00,"{'url_name': 'mesa_prime_blueprint', 'pt': {'i...",5.0,1,5f348c8446b8f1014d9c0612,NaN
4,True,en,pc,"{'region': 'en', 'reputation': 2, 'ingame_name...",sell,2020-08-13T00:42:36.966+00:00,2020-08-13T00:42:36.966+00:00,"{'url_name': 'tranquil_cleave', 'pt': {'item_n...",5.0,1,5f348c7c58c4d0019347a7c9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
427,True,en,pc,"{'region': 'en', 'reputation': 4, 'ingame_name...",sell,2020-08-13T00:16:17.192+00:00,2020-08-13T00:16:17.192+00:00,"{'url_name': 'explosive_legerdemain', 'pt': {'...",24.0,1,5f348651ad961e01bd2d3b0a,3.0
428,True,en,pc,"{'region': 'en', 'reputation': 4, 'ingame_name...",sell,2020-08-13T00:16:16.219+00:00,2020-08-13T00:16:16.219+00:00,"{'url_name': 'mag_prime_systems', 'pt': {'item...",13.0,1,5f34865086284f017326d8a8,NaN
429,True,en,pc,"{'region': 'en', 'reputation': 0, 'ingame_name...",sell,2020-08-13T00:16:15.278+00:00,2020-08-13T00:16:15.278+00:00,"{'url_name': 'soma_prime_set', 'pt': {'item_na...",40.0,1,5f34864f58304b015604b0d3,NaN
430,True,en,pc,"{'region': 'en', 'reputation': 1, 'ingame_name...",sell,2020-08-13T00:16:14.812+00:00,2020-08-13T00:16:14.812+00:00,"{'url_name': 'adaptation', 'pt': {'item_name':...",99.0,1,5f34864e58c4d001876ad152,10.0
